In [1]:
from zemberek import TurkishMorphology, TurkishSentenceNormalizer
from TurkishStemmer import TurkishStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import pandas as pd
import string
import emoji
import nltk
import re

In [2]:
data = pd.read_csv('../dataset/tweets.csv')
data

,text,sexist
0,tecavüze uğrayan kadınlar saatte dışarda işi v...,1.0
1,küçük turistik bir sahil kasabasında doğup büy...,1.0
2,erkek adam ağlamaz cümlesi yerine ağla adam ol...,1.0
3,size saygısızlık yapan insanı muhattap almak a...,1.0
4,karı ağlama artık,1.0
...,...,...
2625,kadın gitti karşı kaldırımda çöp olmasını sall...,0.0
2626,kadın market alışverişi için evden çıktığı sır...,0.0
2627,Kadın aç olduğunu söylüyor diğer kadın da para...,0.0
2628,Kadın hakları savunucuları neredesiniz,0.0


In [4]:
def duzelt_yazi_zemberek(yazi):
    # Zemberek nesnelerini oluştur
    morphology = TurkishMorphology.create_with_defaults()
    normalizer = TurkishSentenceNormalizer(morphology)
    # Cümleleri normalize et
    duzeltilmis_yazi = normalizer.normalize(yazi)
    return duzeltilmis_yazi

In [5]:
stop_words_list = {'a','acaba','altı','altmış','ama','ancak','arada','artık','asla','aslında','ayrıca','az','bana','bazen','bazı''bazıları',
'belki','ben','benden','beni','benim','beri','beş','bile','bilhassa','bin','bir','biraz','birçoğu','birçok','biri','birisi','birkaç',
'birşey','biz','bizden','bize','bizi','bizim','böyle','böylece','bu','buna','bunda','bundan','bunlar','bunları','bunların','bunu',
'bunun','burada','bütün','çoğu','çoğunu','çok','çünkü','da','daha','dahi','dan','de','defa','değil','diğer','diğeri','diğerleri','diye',
'doksan','dokuz','dolayı','dolayısıyla','dört','ee','edecek','eden','ederek','edilecek','ediliyor','edilmesi','ediyor','eğer','elbette',
'elli','en','etmesi','etti','ettiği','ettiğini','fakat','falan','filan','gene','gereği','gerek','gibi','göre','hala','halde','halen','hangi','hangisi',
'hani','hatta','hem','henüz','hep','hepsi','her','herhangi','herkes','herkese','herkesi','herkesin','hiç','hiçbir','hiçbiri','için','içinde',
'iki','ile','ilgili','ise','işte','itibaren','itibariyle','kaç','kadar','karşın','kendi','kendilerine','kendine','kendini','kendisi','kendisine','kendisini','kez','ki',
'kim','kime','kimi','kimin','kimisi','kimse','kırk','madem','mi','mı','milyar','milyon','mu','mü','nasıl','ne','neden','nedenle','nerde','nerede',
'nereye','neyse','niçin','nin','nın','niye','nun','nün','öbür','olan','olarak','oldu','olduğu','olduğunu','olduklarını','olmadı','olmadığı','olmak',
'olması','olmayan','olmaz','olsa','olsun','olup','olur','olur','olursa','oluyor','on','ön','ona','önce','ondan','onlar','onlara','onlardan',
'onları','onların','onu','onun','orada','öte','ötürü','otuz','öyle','oysa','pek','rağmen','sana','sanki','sanki','şayet','şekilde','sekiz','seksen','sen','senden','seni','senin','şey',
'şeyden','şeye','şeyi','şeyler','şimdi','siz','siz','sizden','size','sizi','sizin','sizin','sonra','şöyle','şu'
,'şuna','şunları','şunu','ta','tabii','tam','tamam','tamamen','tarafından','trilyon','tüm','tümü','üç','un','ün','üzere','var','vardı','ve','veya',
'ya','yani','yapacak','yapılan','yapılması','yapıyor','yapmak','yaptı','yaptığı','yaptığını','yaptıkları','ye','yedi','yetmiş','yi','yı','yine','yirmi','yoksa','yu','yüz','zaten','zira'}


In [6]:

def veri_temizleme(text):
    # Küçük harfe çevirme
    text = text.lower()
    #kesme işareti ve sonrasındaki karakterlerin kaldırılması
    text = re.sub("'(\w+)","",text)
    text = re.sub("[“,`,',’,\",❞]"," ",text)
    # Noktalama işaretlerini kaldırma
    translator = str.maketrans('', '', string.punctuation)
    text = text.translate(translator)
    #Yeni satır karakterleri, boşluk karakteri ile değiştirme
    text = text.replace("\\n"," ")
    # URL'leri kaldırma
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    # Sayıları kaldırma
    text = re.sub(r'\d+', '', text)
    # Kullanıcı adlarını kaldırma
    text = re.sub(r'@\S+', '', text)
    # Hashtag'leri kaldırma
    text = re.sub(r'#\S+', '', text)
    # Emoji'leri kaldırma
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F700-\U0001F77F"  # alchemical symbols
                           u"\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
                           u"\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
                           u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
                           u"\U0001FA00-\U0001FA6F"  # Chess Symbols
                           u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
                           u"\U00002702-\U000027B0"  # Dingbats
                           u"\U000024C2-\U0001F251" 
                           "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)
    # Stopwords'leri kaldırma
    stop_words = set(stop_words_list)
    words = word_tokenize(text)
    words = [word for word in words if word not in stop_words]
    # Tekrar eden boşlukları kaldırma
    text = ' '.join(words)
    #text = duzelt_yazi_zemberek(text)
    return text

In [7]:

for index,row in data.iterrows() : 
  data['text'][index] = veri_temizleme(data['text'][index])

data
#  row['text'] = veri_temizleme(row['text'])

,text,sexist
0,tecavüze uğrayan kadınlar saatte dışarda işi v...,1.0
1,küçük turistik sahil kasabasında doğup büyüdüm...,1.0
2,erkek adam ağlamaz cümlesi yerine ağla adam ol...,1.0
3,saygısızlık yapan insanı muhattap almak adam y...,1.0
4,karı ağlama,1.0
...,...,...
2625,kadın gitti karşı kaldırımda çöp olmasını sall...,0.0
2626,kadın market alışverişi evden çıktığı sırada y...,0.0
2627,kadın aç söylüyor kadın para veriyor sonrasınd...,0.0
2628,kadın hakları savunucuları neredesiniz,0.0


In [8]:
data['text'][0]

'tecavüze uğrayan kadınlar saatte dışarda işi varmışdan iğrenç yaklaşım varsa budur çocuğun okul yolunda işi varmış sokaklar köpeklerin dışarı çıkmak haddine haketmiş zaman hay oraya çıkaranlara'

In [9]:
data

,text,sexist
0,tecavüze uğrayan kadınlar saatte dışarda işi v...,1.0
1,küçük turistik sahil kasabasında doğup büyüdüm...,1.0
2,erkek adam ağlamaz cümlesi yerine ağla adam ol...,1.0
3,saygısızlık yapan insanı muhattap almak adam y...,1.0
4,karı ağlama,1.0
...,...,...
2625,kadın gitti karşı kaldırımda çöp olmasını sall...,0.0
2626,kadın market alışverişi evden çıktığı sırada y...,0.0
2627,kadın aç söylüyor kadın para veriyor sonrasınd...,0.0
2628,kadın hakları savunucuları neredesiniz,0.0


In [10]:
data.to_csv("../dataset/cleaned_Data.csv", header=True, index=False)